In [94]:
import pandas as pd
import numpy as np 
from sklearn.ensemble import RandomForestClassifier


In [95]:
dp = pd.read_csv('train.csv')
age = dp['Age']
age[~age.isnull()]
#age.describe() #decribe column for statiscial value
no_null_set = dp.dropna()
Y = no_null_set['Survived']
X = no_null_set[no_null_set.columns.difference(['Survived'])]

#female is 0; male is 1
train_data = pd.concat([X,pd.get_dummies(X['Sex'])], axis = 1)
train_data = train_data[train_data.columns.difference(['Embarked','Sex','Name','female','Cabin','Ticket'])]
train_data = train_data.rename(columns=lambda x: x.replace('male', 'Sex'))

In [96]:
model = RandomForestClassifier(n_estimators = 100)
model.fit(train_data,Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [97]:
# preparing testing data
test_data = pd.concat([dp[dp.columns.difference(['Survived'])],pd.get_dummies(dp['Sex'])], axis = 1)
test_data = test_data[test_data.columns.difference(['Embarked','Sex','Name','female','Cabin','Ticket'])]
test_data = test_data.rename(columns=lambda x: x.replace('male', 'Sex'))
test_data = test_data.fillna(-1)
test_result = dp['Survived']

In [98]:
from sklearn.metrics import accuracy_score
accuracy_score(test_result, model.predict(test_data))

0.64983164983164987

In [99]:
test = pd.read_csv('test.csv')
test = pd.concat([test,pd.get_dummies(test['Sex'])], axis = 1)
test = test[test.columns.difference(['Embarked','Sex','Name','female','Cabin','Ticket'])]
no_null_test = test.dropna()
prediction = model.predict(no_null_test)
prediction = {'PassengerId': no_null_test['PassengerId'], 'Survived': prediction}
pd.DataFrame(prediction).to_csv('prediction.csv', index = False)